In [ ]:
%pip install duckdb
%pip install pyarrow
%pip install polars

[DuckDB](https://duckdb.org) is an in-process SQL OLAP database management system designed for fast analytical query processing. It is lightweight, easy to use, and supports various data formats like JSON, Parquet, and CSV. 

I chose [DuckDB](https://duckdb.org/why_duckdb) because it provides high-performance SQL querying capabilities directly within Python, making it ideal for handling and analyzing structured data efficiently in-memory without the need for a separate database server. Its seamless integration with modern data formats and tools like Polars further enhances its utility for data workflows. Additionally, DuckDB can natively read columnar data and JSON hierarchical data, making it highly versatile for complex data structures.

In [1]:
import duckdb as db
import polars as pl

- Create an in-memory DuckDB database
- Load the synthenic raw data in JSON format into the DuckDB database

In [2]:
conn = db.connect()
conn.execute("CREATE TABLE synthetic_raw AS SELECT * FROM read_json_auto('synthetic_raw.json')")

Output the inferred JSON schema.

In [3]:
print(conn.execute("DESCRIBE SELECT * FROM read_json_auto('synthetic_raw.json')").fetchall())

[('Patient ID', 'VARCHAR', 'YES', None, None, None), ('Survival time (days)', 'BIGINT', 'YES', None, None, None), ('Event (death: 1, alive: 0)', 'BIGINT', 'YES', None, None, None), ('Tumor size (cm)', 'DOUBLE', 'YES', None, None, None), ('Grade', 'BIGINT', 'YES', None, None, None), ('Stage (TNM 8th edition)', 'VARCHAR', 'YES', None, None, None), ('Type.Adjuvant', 'VARCHAR', 'YES', None, None, None), ('batch', 'BIGINT', 'YES', None, None, None), ('EGFR', 'VARCHAR', 'YES', None, None, None), ('KRAS', 'VARCHAR', 'YES', None, None, None), ('Usage', 'STRUCT(Cigarette VARCHAR, "Pack per year" DOUBLE)', 'YES', None, None, None), ('Demog', 'STRUCT(Age BIGINT, Sex VARCHAR)', 'YES', None, None, None)]


In [4]:
my_study = "CureAll Rx"

Create an SDTM DM domain dataset as a polars dataframe. Notice how it recognizes JSONPath for object or key selection, e.g., `Demog.Age`.

In [5]:
sdtm_dm = conn.execute(f"""
	SELECT 
		'{my_study}' AS STUDYID,
		'DM' AS DOMAIN,
		"Patient ID" AS USUBJID,
		Demog.Age AS AGE, 
		'YEARS' AS AGEU, 
		CASE Demog.Sex
			WHEN 'Female' THEN 'F' 
			WHEN 'Male' THEN 'M' 
		END AS SEX 
	FROM synthetic_raw
""").pl()

Similarly, make one for SDTM SU.

In [6]:
sdtm_su = conn.execute(f"""
	SELECT 
		'{my_study}' AS STUDYID,
		'SU' AS DOMAIN,
		"Patient ID" AS USUBJID,
		1 AS SUSEQ,
		'CIGARETTES' AS SUTRT,
		Usage."Pack per year" AS SUDOSE,
		'PACK' AS SUDOSU,
		'PER YEAR' AS SUDOSFRQ,
		CASE 
			WHEN Usage.Cigarette = 'Never' THEN 'N' 
		END AS SUOCCUR,
		CASE 
			WHEN Usage.Cigarette = 'Former' THEN 'BEFORE' 
		END AS SUSTRF,
		CASE 
			WHEN Usage.Cigarette = 'Current' THEN 'ONGOING' 
		END AS SUENRF
	FROM synthetic_raw
""").pl()

In [7]:
conn.close()

In [8]:
sdtm_dm.write_csv("sdtm_dm.csv")
sdtm_su.write_csv("sdtm_su.csv")